# 영어 pdf를 한글로 번역해서 txt로 변환

pip install azure-cognitiveservices-vision-computervision

In [2]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import time
import os
import requests

# Azure Computer Vision 설정
subscription_key = "FQPue8cIR514gwydZdn1xstorLwXFohwwXi0morkCwLxybsl7wnyJQQJ99BAACHYHv6XJ3w3AAAFACOGHyEy"
endpoint = "https://team3-computervision.cognitiveservices.azure.com/"

# Azure Translator 설정
translator_key = "BGTBUtPKISzjZjIJhs67n5fc8CJmtFvTv43nGmhjxFuAsEim9HedJQQJ99BAACHYHv6XJ3w3AAAbACOGYLLS"  # Translator 키
translator_endpoint = "https://api.cognitive.microsofttranslator.com"
translator_region = "eastus2"  # Translator 서비스의 지역 (예: "koreacentral")

# Computer Vision Client 생성
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

# PDF 파일 경로 설정
pdf_path = "pdf/Laparoscopic Cholecystectomy-NIH.pdf"  # OCR을 실행할 입력 파일
input_file_name = os.path.basename(pdf_path).split('.')[0]  # 파일 이름 추출 (확장자 제외)

# 결과 파일 경로 설정
results_folder = "results"  # 결과 파일을 저장할 폴더 이름
os.makedirs(results_folder, exist_ok=True)  # 폴더가 없으면 생성
output_file = os.path.join(results_folder, f"{input_file_name}.txt")  # 저장 파일 경로
translated_file = os.path.join(results_folder, f"{input_file_name}_translated.txt")  # 번역 결과 파일

# PDF를 OCR 처리 (Read API)
with open(pdf_path, "rb") as pdf_file:
    read_response = computervision_client.read_in_stream(pdf_file, raw=True)

# Operation ID 가져오기
operation_location = read_response.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# OCR 처리 상태 확인
print("OCR 처리 중...")
while True:
    result = computervision_client.get_read_result(operation_id)
    if result.status not in [OperationStatusCodes.running, OperationStatusCodes.not_started]:
        break
    time.sleep(1)

# 번역 함수 정의
def translate_text(text, to_language="ko"):
    path = "/translate"
    constructed_url = translator_endpoint + path

    headers = {
        "Ocp-Apim-Subscription-Key": translator_key,
        "Ocp-Apim-Subscription-Region": translator_region,
        "Content-Type": "application/json"
    }

    body = [{"text": text}]

    response = requests.post(constructed_url, headers=headers, json=body, params={"api-version": "3.0", "to": to_language})
    response.raise_for_status()

    translations = response.json()
    return translations[0]['translations'][0]['text']

# 결과 저장 및 번역
if result.status == OperationStatusCodes.succeeded:
    with open(output_file, "w", encoding="utf-8-sig") as file:
        with open(translated_file, "w", encoding="utf-8-sig") as t_file:
            for page in result.analyze_result.read_results:
                for line in page.lines:
                    original_text = line.text
                    translated_text = translate_text(original_text)

                    file.write(original_text + "\n")  # 원본 텍스트 저장
                    t_file.write(translated_text + "\n")  # 번역 텍스트 저장

    print(f"OCR 결과가 '{output_file}' 파일에 저장되었습니다!")
    print(f"번역 결과가 '{translated_file}' 파일에 저장되었습니다!")
else:
    print("OCR 처리가 실패하였습니다.")

OCR 처리 중...
OCR 결과가 'results\Laparoscopic Cholecystectomy-NIH.txt' 파일에 저장되었습니다!
번역 결과가 'results\Laparoscopic Cholecystectomy-NIH_translated.txt' 파일에 저장되었습니다!


# medical_terms.json에 저장해 놓은 의학용어 맵핑 적용해서 번역하기
# 파일명 원래 파일명으로 유지, 중복일 경우 뒤에 숫자 붙여서 저장하는 로직
# 이미지도 OCR/trans도 추가 : img 폴더 내의 모든 이미지를 한꺼번에 처리

In [3]:
import json
import requests
import time
import os
import re
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials

# Azure Computer Vision 설정
subscription_key = "FQPue8cIR514gwydZdn1xstorLwXFohwwXi0morkCwLxybsl7wnyJQQJ99BAACHYHv6XJ3w3AAAFACOGHyEy"
endpoint = "https://team3-computervision.cognitiveservices.azure.com/"

# Azure Translator 설정
translator_key = "BGTBUtPKISzjZjIJhs67n5fc8CJmtFvTv43nGmhjxFuAsEim9HedJQQJ99BAACHYHv6XJ3w3AAAbACOGYLLS"
translator_endpoint = "https://api.cognitive.microsofttranslator.com"
translator_region = "eastus2"

# JSON 파일에서 딕셔너리 불러오기
def load_dictionary(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# Preprocessing: 단어를 태그로 감싸기
def preprocess_text(text, dictionary):
    for eng_term, translations in dictionary.items():
        if eng_term in text:
            text = text.replace(eng_term, f"[[{translations[0]}]]")  # 태그로 감싸기
    return text

# Postprocessing: 태그 제거
def apply_postprocessing(text, dictionary):
    text = re.sub(r"\[\[(.*?)\]\]", r"\1", text)  # 태그 제거
    for eng_term, translations in dictionary.items():
        for translation in translations:
            if translation in text:
                text = text.replace(translation, translations[0])  # 첫 번째 번역 사용
    return text

# 번역 함수: Preprocessing + Translate + Postprocessing
def translate_text_with_preprocessing(text, dictionary, to_language="ko"):
    preprocessed_text = preprocess_text(text, dictionary)  # Preprocessing 단계
    path = "/translate"
    constructed_url = translator_endpoint + path

    headers = {
        "Ocp-Apim-Subscription-Key": translator_key,
        "Ocp-Apim-Subscription-Region": translator_region,
        "Content-Type": "application/json"
    }

    body = [{"text": preprocessed_text}]
    response = requests.post(constructed_url, headers=headers, json=body, params={"api-version": "3.0", "to": to_language})
    response.raise_for_status()

    translations = response.json()
    translated_text = translations[0]['translations'][0]['text']
    
    return apply_postprocessing(translated_text, dictionary)  # Postprocessing 단계

# 파일명이 겹칠 경우 새로운 파일명 생성
def generate_unique_filename(folder, base_name):
    count = 1
    base, ext = os.path.splitext(base_name)
    new_name = base_name
    while os.path.exists(os.path.join(folder, new_name)):
        new_name = f"{base}({count}){ext}"
        count += 1
    return os.path.join(folder, new_name)

def process_pdf_and_translate(pdf_path, results_folder, dictionary_file):
    os.makedirs(results_folder, exist_ok=True)
    dictionary = load_dictionary(dictionary_file)

    input_filename = os.path.basename(pdf_path).split('.')[0]
    ocr_output_filename = f"{input_filename}_OCR.txt"
    translated_output_filename = f"{input_filename}_Translated.txt"

    # 중복 확인 및 파일명 업데이트
    ocr_output_file = generate_unique_filename(results_folder, ocr_output_filename)
    translated_output_file = generate_unique_filename(results_folder, translated_output_filename)

    # Computer Vision Client 생성
    computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

    # OCR 처리 (PDF 파일을 Azure Computer Vision API에 업로드)
    with open(pdf_path, "rb") as pdf_file:
        read_response = computervision_client.read_in_stream(pdf_file, raw=True)

    operation_location = read_response.headers["Operation-Location"]
    operation_id = operation_location.split("/")[-1]

    print(f"{pdf_path} OCR 처리 중...")
    while True:
        result = computervision_client.get_read_result(operation_id)
        if result.status not in [OperationStatusCodes.running, OperationStatusCodes.not_started]:
            break
        time.sleep(1)

    # OCR 결과 저장 및 번역
    if result.status == OperationStatusCodes.succeeded:
        with open(ocr_output_file, "w", encoding="utf-8") as file:
            with open(translated_output_file, "w", encoding="utf-8") as t_file:
                for page in result.analyze_result.read_results:
                    for line in page.lines:
                        original_text = line.text
                        translated_text = translate_text_with_preprocessing(original_text, dictionary)

                        file.write(original_text + "\n")  # OCR 원본 저장
                        t_file.write(translated_text + "\n")  # 번역 저장

        print(f"OCR 결과가 '{ocr_output_file}' 파일에 저장되었습니다!")
        print(f"번역 결과가 '{translated_output_file}' 파일에 저장되었습니다!")
    else:
        print(f"{pdf_path} OCR 처리가 실패했습니다.")


# 이미지 OCR 및 번역 처리
def process_image_and_translate(image_path, results_folder, dictionary):
    os.makedirs(results_folder, exist_ok=True)
    dictionary = load_dictionary(dictionary)

    input_filename = os.path.basename(image_path).split('.')[0]
    ocr_output_filename = f"{input_filename}_OCR.txt"
    translated_output_filename = f"{input_filename}_Translated.txt"

    # 중복 확인 및 파일명 업데이트
    ocr_output_file = generate_unique_filename(results_folder, ocr_output_filename)
    translated_output_file = generate_unique_filename(results_folder, translated_output_filename)

    # Computer Vision Client 생성
    computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

    # OCR 처리
    with open(image_path, "rb") as image_file:
        read_response = computervision_client.read_in_stream(image_file, raw=True)

    operation_location = read_response.headers["Operation-Location"]
    operation_id = operation_location.split("/")[-1]

    print(f"{image_path} OCR 처리 중...")
    while True:
        result = computervision_client.get_read_result(operation_id)
        if result.status not in [OperationStatusCodes.running, OperationStatusCodes.not_started]:
            break
        time.sleep(1)

    if result.status == OperationStatusCodes.succeeded:
        with open(ocr_output_file, "w", encoding="utf-8") as file:
            with open(translated_output_file, "w", encoding="utf-8") as t_file:
                for page in result.analyze_result.read_results:
                    for line in page.lines:
                        original_text = line.text
                        translated_text = translate_text_with_preprocessing(original_text, dictionary)  # 번역 실행
                        file.write(original_text + "\n")
                        t_file.write(translated_text + "\n")

        print(f"OCR 결과가 '{ocr_output_file}' 파일에 저장되었습니다!")
        print(f"번역 결과가 '{translated_output_file}' 파일에 저장되었습니다!")
    else:
        print(f"{image_path} OCR 처리가 실패했습니다.")

# PDF 및 이미지 처리 및 번역 실행
def process_files_and_translate(pdf_path, image_folder, results_folder, dictionary_file):
    os.makedirs(results_folder, exist_ok=True)

    # PDF 처리
    if pdf_path:
        process_pdf_and_translate(pdf_path, results_folder, dictionary_file)

    # 이미지 처리
    if image_folder:
        for image_file in os.listdir(image_folder):
            image_path = os.path.join(image_folder, image_file)
            if os.path.isfile(image_path):
                process_image_and_translate(image_path, results_folder, dictionary_file)

# Main 실행
if __name__ == "__main__":
    pdf_path = "pdf/Laparoscopic Cholecystectomy-NIH.pdf"  # OCR 실행할 PDF 파일 경로
    image_folder = "img"  # OCR 실행할 이미지 폴더 경로
    results_folder = "results"  # 결과 저장 폴더
    dictionary_file = "medical_terms.json"  # 딕셔너리 파일 경로
    process_files_and_translate(pdf_path, image_folder, results_folder, dictionary_file)


pdf/Laparoscopic Cholecystectomy-NIH.pdf OCR 처리 중...
OCR 결과가 'results\Laparoscopic Cholecystectomy-NIH_OCR.txt' 파일에 저장되었습니다!
번역 결과가 'results\Laparoscopic Cholecystectomy-NIH_Translated.txt' 파일에 저장되었습니다!
img\K-013.jpg OCR 처리 중...
OCR 결과가 'results\K-013_OCR.txt' 파일에 저장되었습니다!
번역 결과가 'results\K-013_Translated.txt' 파일에 저장되었습니다!
